In [63]:
# var 3

import random

from collections import OrderedDict
from itertools import islice

x*y*z+x*y*z^2+y+u*x*y^2*z^2+w^2*x*y^2*z=-50													

In [64]:
POPULATION_SIZE = 3000
MIN_VALUE = -300
MAX_VALUE = 300
NUM_ITEMS_FOR_SELECTION = 1000
NUM_ITEMS_FOR_MUTATION = 500
NUM_OF_SUITABLE_DESCEDANTS = 200

MUTATION_PROBABILITY_FOR_SUITABLE = 0.01
MUTATION_PROBABILITY_FOR_UNSUITABLE = 0.25

In [65]:
def generate_item(min_value, max_value):
    return [random.randint(min_value, max_value) for _ in ['u', 'w', 'x', 'y', 'z']]

In [66]:
def create_initial_population(population_size: int, min_value: int, max_value: int):
    population = []
    for index in range(population_size):
        item = generate_item(min_value, max_value)
        population.append(item)
    return population

In [67]:
population = create_initial_population(POPULATION_SIZE, MIN_VALUE, MAX_VALUE)

In [68]:
TARGET_VALUE = -50


def target_function(u, w, x, y, z):
    return abs(x*y*z + x*y*z**2 + y + u*x*y**2*z**2 + w**2*x*y**2*z - TARGET_VALUE)

In [69]:
def perform_tournament_selection(population, num_items_for_selection, num_items_for_mutation, target_function):
    sample =  random.sample(population, num_items_for_selection)
    
    target_item_dict = {
        target_function(*item): item
        for item in sample
    }
    ordered_target_item_dict = OrderedDict(sorted(target_item_dict.items()))
    
    return list(OrderedDict(islice(ordered_target_item_dict.items(), 0, num_items_for_mutation)).values())

In [70]:
selection = perform_tournament_selection(population, NUM_ITEMS_FOR_SELECTION, NUM_ITEMS_FOR_MUTATION, target_function)

In [71]:
def perform_homogeneous_crossing_for_two_elements(first_item, second_item):
    new_item = [
        random.choice([coordinate1, coordinate2]) for coordinate1, coordinate2 in zip(first_item, second_item)
    ]
    return new_item


In [72]:
def perform_homogeneous_crossing(selection, descendants_size):
    descendants = [
        perform_homogeneous_crossing_for_two_elements(random.choice(selection), random.choice(selection))
        for _ in range(descendants_size)
    ]
    return descendants

In [73]:
def should_perform_coordinate_mutation(mutation_probability):
    return random.random() < mutation_probability

In [74]:
def perform_mutation(
   selection, num_of_suitable_descendants, mutation_probability_for_suitable, mutation_probability_for_unsuitable
):
    
    for item in selection[:num_of_suitable_descendants]:
        for index, coordinate in enumerate(item):
            if should_perform_coordinate_mutation(mutation_probability_for_suitable):
                item[index] = random.randint(MIN_VALUE, MAX_VALUE)
    
    for item in selection[num_of_suitable_descendants:]:
        for index, coordinate in enumerate(item):
            if should_perform_coordinate_mutation(mutation_probability_for_unsuitable):
                item[index] = random.randint(MIN_VALUE, MAX_VALUE)

    return selection

In [75]:
selection = perform_mutation(
    selection,
    NUM_OF_SUITABLE_DESCEDANTS,
    MUTATION_PROBABILITY_FOR_SUITABLE,
    MUTATION_PROBABILITY_FOR_UNSUITABLE
)

In [76]:
def perform_descendants_substitution(selection, population_size, target_function):
    return perform_tournament_selection(selection, len(selection), population_size, target_function)

In [77]:
perform_descendants_substitution(selection, POPULATION_SIZE, target_function)[:5]

[[12, 73, -86, -39, 0],
 [-146, 285, -184, 0, -69],
 [-109, -11, -97, 2, 12],
 [82, 119, 220, -2, -176],
 [218, -48, 107, 14, -1]]

In [78]:
def run_genetic_algorithm(
    population_size, min_value, max_value, num_items_for_selection, num_items_for_mutation, target_function,
    num_of_suitable_descedants, mutation_probability_for_suitable, mutation_probability_for_unsuitable
):
    initial_population = create_initial_population(population_size, min_value, max_value)
    iteration = 0
    while True:
        selection = perform_tournament_selection(
            initial_population, num_items_for_selection, num_items_for_mutation, target_function
        )
        descendants = perform_homogeneous_crossing(selection, 2*population_size)
        descedants = perform_mutation(
            descendants,
            num_of_suitable_descedants,
            mutation_probability_for_suitable,
            mutation_probability_for_unsuitable
        )
        selection = perform_descendants_substitution(
            descendants, population_size, target_function
        )
        
        target_values = [target_function(*item) for item in selection]
        print(f"MIN of target values: {min(target_values)}; AVG of target values: {sum(target_values)/len(target_values)}")
        
        result = []
        for item in selection:
            if target_function(*item) == 0:
                result.append(item)
        if result:
            return result
            
        if iteration == 10000:
            return None
        iteration += 1

In [79]:
result = run_genetic_algorithm(
    POPULATION_SIZE,
    MIN_VALUE,
    MAX_VALUE,
    NUM_ITEMS_FOR_SELECTION,
    NUM_ITEMS_FOR_MUTATION,
    target_function,
    NUM_OF_SUITABLE_DESCEDANTS,
    MUTATION_PROBABILITY_FOR_SUITABLE,
    MUTATION_PROBABILITY_FOR_UNSUITABLE
)

MIN of target values: 50; AVG of target values: 322004340855.6047
MIN of target values: 16; AVG of target values: 293214926092.82367
MIN of target values: 1; AVG of target values: 355333658852.5917
MIN of target values: 6; AVG of target values: 345018900704.7003
MIN of target values: 6; AVG of target values: 326391706884.4553
MIN of target values: 4; AVG of target values: 359708157280.524
MIN of target values: 43; AVG of target values: 387722701092.3233
MIN of target values: 6; AVG of target values: 396318571235.3173
MIN of target values: 22; AVG of target values: 311928363392.398
MIN of target values: 5; AVG of target values: 313805782306.183
MIN of target values: 5; AVG of target values: 350742004189.66766
MIN of target values: 24; AVG of target values: 397780258866.53766
MIN of target values: 17; AVG of target values: 398832240956.3077
MIN of target values: 15; AVG of target values: 379476842504.07666
MIN of target values: 1; AVG of target values: 297032764711.6197
MIN of target val

In [80]:
result

[[-188, 216, 0, -50, -35]]

z+w*x^2*z+u*x*y^2*z+u*w^2*x^2*z^2+w*x^2*y^2=-50 

In [81]:
TARGET_VALUE = -50


def target_function(u, w, x, y, z):
    return abs(z + w*x**2*z + u*x*y**2*z + u*w**2*x**2*z**2 + w*x**2*y**2 - TARGET_VALUE)

In [82]:
result = run_genetic_algorithm(
    POPULATION_SIZE,
    MIN_VALUE,
    MAX_VALUE,
    NUM_ITEMS_FOR_SELECTION,
    NUM_ITEMS_FOR_MUTATION,
    target_function,
    NUM_OF_SUITABLE_DESCEDANTS,
    MUTATION_PROBABILITY_FOR_SUITABLE,
    MUTATION_PROBABILITY_FOR_UNSUITABLE
)

MIN of target values: 51; AVG of target values: 7259233372816.644
MIN of target values: 7; AVG of target values: 5998004251515.987
MIN of target values: 15; AVG of target values: 8402131802951.565
MIN of target values: 2; AVG of target values: 7132263747130.533
MIN of target values: 17; AVG of target values: 7363223789705.213
MIN of target values: 26; AVG of target values: 7008498256372.1045
MIN of target values: 7; AVG of target values: 8433149565466.428
MIN of target values: 7; AVG of target values: 7368471965174.517
MIN of target values: 18; AVG of target values: 9438000182404.088
MIN of target values: 7; AVG of target values: 6789505382694.113
MIN of target values: 18; AVG of target values: 8417748247727.553
MIN of target values: 7; AVG of target values: 8196888582273.912
MIN of target values: 7; AVG of target values: 7582242303474.738
MIN of target values: 6; AVG of target values: 8381821082571.358
MIN of target values: 14; AVG of target values: 7019396347546.314
MIN of target val

In [83]:
result

[[-245, 210, 0, 66, -50]]

u^2*w^2*x^2*z+u^2*x*y^2+w*x*y+y+u^2*x^2*y^2=40

In [84]:
TARGET_VALUE = 40


def target_function(u, w, x, y, z):
    return abs(u**2*w**2*x**2*z + u**2*x*y**2 + w*x*y + y + u**2*x**2*y**2 - TARGET_VALUE)

In [85]:
result = run_genetic_algorithm(
    POPULATION_SIZE,
    MIN_VALUE,
    MAX_VALUE,
    NUM_ITEMS_FOR_SELECTION,
    NUM_ITEMS_FOR_MUTATION,
    target_function,
    NUM_OF_SUITABLE_DESCEDANTS,
    MUTATION_PROBABILITY_FOR_SUITABLE,
    MUTATION_PROBABILITY_FOR_UNSUITABLE
)

MIN of target values: 1; AVG of target values: 7812913988794.492
MIN of target values: 88; AVG of target values: 8240554663111.71
MIN of target values: 5; AVG of target values: 9919231421212.588
MIN of target values: 15; AVG of target values: 10881155683235.297
MIN of target values: 83; AVG of target values: 10122221212253.854
MIN of target values: 25; AVG of target values: 10501126465703.234
MIN of target values: 12; AVG of target values: 7897746652254.378
MIN of target values: 30; AVG of target values: 9401030233693.576
MIN of target values: 9; AVG of target values: 8961607162046.738
MIN of target values: 23; AVG of target values: 10682184983923.535
MIN of target values: 13; AVG of target values: 10101648198762.002
MIN of target values: 28; AVG of target values: 10219051314521.434
MIN of target values: 11; AVG of target values: 11070337251464.52
MIN of target values: 8; AVG of target values: 9239747567495.812
MIN of target values: 40; AVG of target values: 10876912818959.074
MIN of t

In [86]:
result

[[-290, 293, 0, 40, 297]]